In [83]:
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfparser import PDFParser
from pdfminer.converter import PDFPageAggregator
from io import StringIO
import os
import codecs
import re 
import camelot
import pandas as pd
import numpy as np
from datetime import datetime,date,timedelta

sectors = ['Livelihoods and basic needs', 'Protection, Gender & Inclusion', 'Health', 'WASH', 'Migration']

# map sectors to naming convention used in "IFRC GO Matrix"
sector_map = {'Livelihoods and basic needs': 'Livelihoods and basic needs',
              'WASH': 'WASH',
              'Health': 'Health',
              'Protection, Gender & Inclusion': 'PGI',
              'Migration': 'Migration'}

keyword_map = {'CEA': 'CEA',
             'RCCE': 'CEA',
             'DRR': 'DRR',
             'education': 'Education',
             'PGI': 'PGI',
             'shelter': 'Shelter',
             'WASH': 'WASH',
             'NSD': 'NSD',
             'migration': 'Migration'}

In [36]:
def ExtractTxtAndTables():
    for country in os.listdir('Approved Plans'):
#         # Test only on Cook Islands
#         if country != 'Cook Islands':
#             continue

        for file in os.listdir('Approved Plans/'+country):

            if file.startswith('MDR') and file.endswith('pdf'):
                output_string = StringIO()
                with open('Approved Plans/'+country+'/'+file, 'rb') as fh:
                    
                    # parse pages with landscape orientation
                    parser = PDFParser(fh)
                    doc = PDFDocument(parser)
                    rsrcmgr = PDFResourceManager()
                    device = TextConverter(rsrcmgr, output_string, laparams=LAParams())
                    interpreter = PDFPageInterpreter(rsrcmgr, device)
                    landscapepages = list()
                    for ix, page in enumerate(PDFPage.create_pages(doc)):
                        # only pages with landscape orientation
                        if page.cropbox[2] < 800:
                            continue
                        interpreter.process_page(page)
                        # count landscape page 
                        landscapepages.append(ix+1)
                    
                    landscapepages = str(landscapepages)
                    landscapepages = landscapepages[1:-1]
                    
                    # find all tables in the landscape pages and create excel tables
                    foundtables = camelot.read_pdf('Approved Plans/'+country+'/'+file, pages=landscapepages)
                    for i in range(len(foundtables)):
                        foundtables[i].to_excel('Approved Plans/'+country+'/'+'Table_test'+str(i)+'.xlsx')
                    
                    # save text of the landscape pages in txt
                    file1 = codecs.open('Approved Plans/'+country+'/'+file.split('.')[0]+'.txt', "w", "utf-8")  # write mode
                    file1.write(output_string.getvalue())
                    file1.close()
                    
                    # parse first page and save corresponding text in txt
                    parser = PDFParser(fh)
                    doc = PDFDocument(parser)
                    rsrcmgr = PDFResourceManager()
                    device = TextConverter(rsrcmgr, output_string, laparams=LAParams())
                    interpreter = PDFPageInterpreter(rsrcmgr, device)
                    landscapepages = list()
                    for ix, page in enumerate(PDFPage.create_pages(doc)):
                        if ix>0:
                            continue
                        interpreter.process_page(page)
                        
                    file1 = codecs.open('Approved Plans/'+country+'/'+file.split('.')[0]+'_firstpage.txt', "w", "utf-8")  # write mode
                    file1.write(output_string.getvalue())
                    file1.close()

In [230]:
def ParseTablesExtractActivities():
    
    for country in os.listdir('Approved Plans'):
#         # Test only on Cook Islands
#         if country != 'Cook Islands':
#             continue

        # merge all tables
        table_all = pd.DataFrame()
        for table in [x for x in os.listdir('Approved Plans/'+country) if x.endswith('xlsx')]:
                table = pd.read_excel('Approved Plans/'+country+'/'+table)
                table_all = pd.concat([table_all, table])
        ncolraw = len(table_all.columns)
        table_all = table_all.rename(columns={0: 'code', 1: 'text'})
        table_all = table_all.reset_index()
        
        if table_all.empty:
            print(country, 'missing tables')
            continue

        # specify sector in each table entry
        table_all['sector'] = ''
        sector_active = ''
        for ix, row in table_all.iterrows():
            for sector in sectors:
                search = sector + ' Output'
                try:
                    if search.lower() in row.text.lower() and 'P&B' in row.code:
                        sector_active = sector
                    else:
                        continue
                except:
                    pass
            table_all.at[ix, 'sector'] = sector_active
        # print results
#         for sector in table_all.sector.unique():
#             print(table_all[table_all.sector == sector].head())

        # get activities
        table_activities = table_all.dropna(subset=['code'])
        table_activities = table_activities[table_activities.code.str.contains('AP')]
        
        # if some activities don't have a sector, drop
        table_activities = table_activities[table_activities['sector']!='']
        
        if table_activities.empty:
            print('ERROR ({}): no activities found, skipping country'.format(country))
            continue
            
        table_activities.text = table_activities.text.apply(lambda x: re.sub(r'\nx', '', str(x)))
        table_activities.text = table_activities.text.apply(lambda x: re.sub(r'\n', ' ', str(x)))
        
        # get activity date (month number)
        table_activities['activity_start'] = 0
        table_activities['activity_end'] = 0
        for ix, row in table_activities.iterrows():
            activity_start, activity_end = 0, 0
            for i in range(2, ncolraw-1):
                if isinstance(row[i], str) and 'x' in row[i]:
                    if activity_start == 0:
                        activity_start = i-1
                    activity_end = i-1
            table_activities.at[ix, 'activity_start'] = activity_start
            table_activities.at[ix, 'activity_end'] = activity_end

        # load output_data
        output_data = pd.read_excel('matrix_filled.xlsx')

        # fill output_data
        for ix, row in table_activities.iterrows():
            data_entry = pd.Series({'Country': country,
                                    sector_map[row['sector']]: row['text'],
                                    'Planned activity start date': row['activity_start'],
                                    'Planned activity end date': row['activity_end']
                                   })
            output_data = output_data.append(data_entry, ignore_index=True)
            
            for key, target in keyword_map.items():
                if key in row['text'] and target != sector_map[row['sector']]:
                    data_entry = pd.Series({'Country': country,
                                            target: row['text'],
                                            'Planned activity start date': row['activity_start'],
                                            'Planned activity end date': row['activity_end']})
                    output_data = output_data.append(data_entry, ignore_index=True)

        # save output data
        output_data.to_excel('matrix_filled.xlsx', index=False)

In [231]:
def ParseTxtExtractExtraInfoSector():
    
    sectors_txt = sectors
    sectors_txt.append('Health and WASH') # some plans (e.g. Philippines) merge the two sectors
    
    for country in os.listdir('Approved Plans'):
#         # Test only on Cook Islands
#         if country != 'Cook Islands':
#             continue

        for file in os.listdir('Approved Plans/'+country):

            if file.startswith('MDR') and file.endswith('txt') and 'firstpage' not in file:
                
                # read txt file
                file1 = codecs.open('Approved Plans/' + country + '/' + file.split('.')[0] + '.txt', "r",
                                    "utf-8")  # write mode
                text = file1.read()
                
                # get detailed operational plan
                text = re.split(r"(.*)Detailed Operational Plan", text, re.MULTILINE | re.DOTALL)[-1]
                
                # loop over sectors and extract extra info
                info_sectors = {}
                
                for sector in sectors_txt:
                    if sector == 'Protection, Gender & Inclusion':
                        sector_re = 'Protection, Gender and Inclusion'
                    elif sector == 'WASH':
                        sector_re = 'Water, sanitation and hygiene'
                    else:
                        sector_re = sector
                        
                    text_sector = re.split(sector_re+' \nPeople', text, re.MULTILINE | re.DOTALL)
                    
                    if len(text_sector)==1:
                        continue
                    text_sector = text_sector[1]
                    
                    for line in re.split(r"\n", text_sector, re.MULTILINE | re.DOTALL)[:10]:
                        if 'targeted:' in line:
                            if re.findall(r"[0-9,]+", line) == []:
                                print(country+' has no target people')
                                peop_target = '-'    
                            else:
                                peop_target = re.findall(r"[0-9,]+", line)[0]
                        if 'Requirements (CHF):' in line:
                            if re.findall(r"[0-9,]+", line) == []:
                                print(country+' has no target people')
                                budget = '-'    
                            else:
                                budget = re.findall(r"[0-9,]+", line)[0]
                            
                    if sector != 'Health and WASH':
                        info_sectors[sector] = {'Number of targeted (if applicable)': peop_target, 'Funding': budget}
                    else:
                        info_sectors['Health'] = {'Number of targeted (if applicable)': peop_target, 'Funding': budget}
                        info_sectors['WASH'] = {'Number of targeted (if applicable)': peop_target, 'Funding': budget}
                    
        # load output_data
        output_data = pd.read_excel('matrix_filled.xlsx')

        # fill output_data
        for sector, infos in info_sectors.items():
            for field, number in infos.items():
                output_data.at[output_data[output_data['Country']==country][~output_data[sector_map[sector]].isna()].index, field] = number

        # save output data
        output_data.to_excel('matrix_filled.xlsx', index=False)

In [232]:


def ParseFirstPage():
    
    for country in os.listdir('Approved Plans'):
#         # Test only on Cook Islands
#         if country != 'Cook Islands':
#             continue

        for file in os.listdir('Approved Plans/'+country):

            if file.startswith('MDR') and file.endswith('.txt') and 'firstpage' not in file:
                
                # read txt file
                file1 = codecs.open('Approved Plans/' + country + '/' + file.split('.')[0] + '_firstpage.txt', "r",
                                    "utf-8")  # write mode
                text = file1.read()
                
                text = text.replace('  ', ' ')
                text = text.replace('\n', ' ')
                text = text.replace(' in '+country, '')
                
                matches = re.findall(r"(?<=Red Cross and Red Crescent Movement partners actively involved in the operation:)(.*)Other partner organizations actively involved in the operation:", text)
                if matches == []:
                    matches = re.findall(r"(?<=Red Cross Red Crescent Movement partners actively involved in the operation:)(.*)Other partner organizations actively involved in the operation:", text)
                if matches == []:
                    matches = re.findall(r"(?<=Red Cross and Red Crescent partners actively involved in the operation:)(.*)Other partner organizations actively involved in the operation:", text)
                
                if matches == []:
                    print(text)
                    print(country, 'none')
                else:
                    RCRC_partners = matches[0]
                    field = 'In country Partner NS'
                    
                # find start date
                matches = re.findall(r"[0-9]{2}/[0-9]{2}/[0-9]{4}", text)
                matches.extend(re.findall(r"[0-9]{2} [A-Z][a-z]+ [0-9]{4}", text))
                if matches == []:
                    print(country, 'no dates found')
                else:
                    dates = []
                    for match in matches:
                        try:
                            dates.append(datetime.strptime(match, '%d/%m/%Y'))
                        except:
                            try:
                                dates.append(datetime.strptime(match, '%d %B %Y'))
                            except:
                                pass
                    dates.sort()
                    # take the most recent one, but not in the future
                    begin_date = dates[-1]
                    i = 2
                    while begin_date > datetime.today():
                        try:
                            begin_date = dates[-i]
                            i += 1
                        except:
                            break
                    
                    # load output_data
                    output_data = pd.read_excel('matrix_filled.xlsx')

                    # fill output_data
                    output_data.at[output_data[output_data['Country']==country].index, field] = RCRC_partners
                    output_data.at[output_data[output_data['Country']==country].index, 'Plan start date'] = begin_date

                    # save output data
                    output_data.to_excel('matrix_filled.xlsx', index=False)

In [233]:
def AdjustDates():
    
    # load output_data
    output_data = pd.read_excel('matrix_filled.xlsx')
    output_data['Plan start date'] = pd.to_datetime(output_data['Plan start date'])
    output_data['Planned activity start date'] = output_data['Planned activity start date'].astype(object)
    output_data['Planned activity end date'] = output_data['Planned activity end date'].astype(object)

    # get plan start date
    for country in output_data.Country.unique():
        for ix, row in output_data[output_data['Country']==country].iterrows():
            plan_start_date = row['Plan start date']
            try:
                activity_start_month_number = int(row['Planned activity start date'])
                activity_end_month_number = int(row['Planned activity end date'])
                activity_start_date = plan_start_date + timedelta(days=(activity_start_month_number-1)*31)
                activity_end_date = plan_start_date + timedelta(days=activity_end_month_number*31)
                output_data.at[ix, 'Planned activity start date'] = str(activity_start_date.date())
                output_data.at[ix, 'Planned activity end date'] = str(activity_end_date.date())
            except:
                output_data.at[ix, 'Planned activity start date'] = ''
                output_data.at[ix, 'Planned activity end date'] = ''
                
    output_data = output_data.drop(columns=['Plan start date'])
            
    # save output data
    output_data.to_excel('matrix_filled.xlsx', index=False)
    

In [237]:
# prepare dataframe
output_data = pd.read_excel('Matrix for data analysis from GO.xlsx')
output_data.drop(output_data.index, inplace=True)
# output_data = pd.read_excel('matrix_filled_temp.xlsx')
output_data.to_excel('matrix_filled.xlsx', index=False)

# do the magic
# ExtractTxtAndTables()
ParseTablesExtractActivities()
ParseTxtExtractExtraInfoSector()
ParseFirstPage()
AdjustDates()

East Asia missing tables
Pacific missing tables
Sth Asia missing tables
Sth East Asia missing tables


C:\ProgramData\Anaconda3\envs\text\lib\site-packages\ipykernel_launcher.py:66: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


Fiji has no target people
Indonesia has no target people
Indonesia has no target people
Indonesia has no target people
Lao PDR has no target people
Lao PDR has no target people
Lao PDR has no target people
Marshall Islands has no target people
Mongolia has no target people
Nepal has no target people
Nepal has no target people
Nepal has no target people
Papua New Guinea has no target people
Philippines has no target people
Solomon Islands has no target people
Timor Leste has no target people
Vanuatu has no target people
Vanuatu has no target people
Viet Nam has no target people
Viet Nam has no target people


In [188]:
# move to output, which is ignored by git, so that we are data responsible
os.rename('matrix_filled.xlsx', 'Output/matrix_filled.xlsx')